In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F
import pickle

In [4]:
# file = open('googleNet_embeddings.pkl', 'rb')
# embeddings = torch.load('../matrix_factorization/InceptionNet_embeddings.pkl')
embeddings = torch.load("../matrix_factorization/sentenceBert_cnn_encoder_decoder.pkl")
embeddings

{'E0': tensor([[-0.1871,  0.2282, -0.1727,  ...,  0.1472,  0.2910, -0.3862],
         [-0.3664,  0.3472, -1.1609,  ..., -0.0622,  0.0587, -0.4521],
         [-0.0877,  0.1031, -0.2516,  ...,  0.2156,  0.2691, -0.1084],
         ...,
         [ 0.2111,  0.6596, -0.1330,  ..., -0.1103,  0.8673, -0.4251],
         [-0.8579,  1.2079, -0.4490,  ..., -0.8057,  0.5456, -1.3085],
         [-0.8488,  0.5984, -0.7300,  ..., -0.3546, -0.1988, -0.0395]],
        requires_grad=True),
 'E1': tensor([[ 0.0860,  0.0018,  0.0369,  ...,  0.0829,  0.0690, -0.0832],
         [ 0.0860,  0.0018,  0.0369,  ...,  0.0829,  0.0690, -0.0832],
         [ 0.0860,  0.0018,  0.0369,  ...,  0.0829,  0.0690, -0.0832],
         ...,
         [ 0.0860,  0.0018,  0.0369,  ...,  0.0829,  0.0690, -0.0832],
         [ 0.0860,  0.0018,  0.0369,  ...,  0.0829,  0.0690, -0.0832],
         [ 0.0860,  0.0018,  0.0369,  ...,  0.0829,  0.0690, -0.0832]],
        requires_grad=True),
 'E2': tensor([[-0.0337, -0.0328, -0.0138,  ...,

In [5]:
train_ids = torch.load("../dataset/train_book_id.pkl")
test_ids = torch.load("../dataset/test_book_id.pkl")
print(train_ids.size())
print(test_ids.size())


RuntimeError: ../dataset/train_book_id.pkl is a zip archive (did you mean to use torch.jit.load()?)

In [6]:
df = pd.read_csv("../dataset/books_with_genres.csv")
df['is_fiction'] = df['binary_category'].apply(lambda x: 1 if x=='fiction' else 0)
df.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,"mystery, thriller, crime",poetry,romance,non-fiction,children,young-adult,"comics, graphic",category,binary_category,is_fiction
0,NaN,7,['189911'],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,False,4.03,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,"fantasy, paranormal",fiction,1
1,0842379428,566,[],US,eng,"[{'count': '6393', 'name': 'to-read'}, {'count...",NaN,False,4.26,B000FCKCJC,...,NaN,NaN,NaN,163.0,NaN,NaN,NaN,non-fiction,non-fiction,0
2,0590417010,193,[],US,eng,"[{'count': '450', 'name': 'to-read'}, {'count'...",NaN,False,4.43,B017RORXNI,...,NaN,NaN,NaN,2.0,109.0,1.0,NaN,children,non-fiction,0
3,1400041694,44,[],US,en-US,"[{'count': '362', 'name': 'to-read'}, {'count'...",NaN,False,3.75,B002OTKEP6,...,NaN,NaN,NaN,37.0,NaN,NaN,NaN,non-fiction,non-fiction,0
4,NaN,1,[],US,eng,"[{'count': '8', 'name': 'to-read'}, {'count': ...",NaN,False,3.83,NaN,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,poetry,non-fiction,0


In [7]:
X_train = embeddings["E0"][0:4000].detach().numpy()
X_test = embeddings["E0"][4000:].detach().numpy()
print(X_train.shape)
print(X_test.shape)

(4000, 50)
(1000, 50)


In [8]:
y_train = df["is_fiction"][0:4000]
y_test = df["is_fiction"][4000:]
print(y_train.shape)
print(y_test.shape)

(4000,)
(1000,)


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score,accuracy_score, precision_score, recall_score
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
score = f1_score(y_test, preds)
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
print('f1_score: ', score)
print('accuarcy: ', accuracy)
print('precision: ', precision)
print('recall: ', recall)

f1_score:  0.7782529016493586
accuarcy:  0.637
precision:  0.637
recall:  1.0


In [10]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(base_estimator=SVC(), n_estimators=10, random_state=0).fit(X_train, y_train)
preds = clf.predict(X_test)
score = f1_score(y_test, preds)
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
print('f1_score: ', score)
print('accuarcy: ', accuracy)
print('precision: ', precision)
print('recall: ', recall)

f1_score:  0.7773677736777367
accuarcy:  0.638
precision:  0.6390293225480284
recall:  0.9921507064364207


In [11]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)
preds = clf.predict(X_test)
score = f1_score(y_test, preds)
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
print('f1_score: ', score)
print('accuarcy: ', accuracy)
print('precision: ', precision)
print('recall: ', recall)

f1_score:  0.7197231833910034
accuarcy:  0.595
precision:  0.6435643564356436
recall:  0.8163265306122449
